In [1]:
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cpi
import pymongo
from tqdm import tqdm
root = "../Dataset/IMDB/"
summary_path = 'IMDB_SUMMARY/SUMMARY_DATA/'
movie_path = 'MOVIES/'

In [2]:
# cpi.update()

In [3]:
limit = 400000

In [4]:
summary = []

for frm in tqdm(range(1, limit, 250)):
    file_name = '{} - {}.json'.format(frm, frm+250-1)
    with open(root + summary_path + file_name, 'r') as f:
        cur_file = json.load(f)
    summary = summary + cur_file

100%|██████████| 1600/1600 [00:56<00:00, 28.29it/s]


In [6]:
len(summary)

399986

In [66]:
# Load data in small chunks
st = 300000 + 1
nd = 400000

In [67]:
movie_list = []

for frm in tqdm(range(st, nd, 250)):
    folder_name = '{} - {}/'.format(frm, frm+250-1)
    movie_files = os.listdir(root + movie_path + folder_name)
    # print("{} -- {}".format(folder_name, len(movie_files)))
    for file_name in movie_files:
        file_path = root + movie_path + folder_name + file_name
        with open(file_path, 'r') as f:
            cur_movie = json.load(f)
        movie_list.append(cur_movie)

100%|██████████| 400/400 [01:03<00:00,  6.29it/s]


In [68]:
type_dict = {}
for movie in movie_list:
    __type = movie['@type']
    if(__type not in type_dict):
        type_dict[__type] = []
    type_dict[__type].append(movie)
for __type in type_dict:
    print(__type, len(type_dict[__type]))

CreativeWork 15378
Movie 65459
TVSeries 14980
VideoGame 874
TVEpisode 15


In [69]:
movie_list = type_dict['Movie']

In [70]:
def get_IMDB_id(movie):
    url = movie['url']
    id = url.split("/")[2]
    return id

In [71]:
for movie in movie_list:
    movie['imdb_id'] = get_IMDB_id(movie)

# Insert data into mongoDB

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["imdb"]

In [3]:
# db.list_collection_names()
movie_collection = db['movies']

In [74]:
for i in tqdm(range(0, len(movie_list))):
    movie_collection.insert_one(movie_list[i])

100%|██████████| 65459/65459 [01:10<00:00, 926.00it/s] 


In [4]:
for movie in movie_collection.find({'imdb_id': 'tt0371746'}).limit(5):
    print(movie)

{'_id': ObjectId('60a6633eef235d48e488a31c'), '@context': 'http://schema.org', '@type': 'Movie', 'url': '/title/tt0371746/', 'name': 'Iron Man', 'image': 'https://m.media-amazon.com/images/M/MV5BMTczNTI2ODUwOF5BMl5BanBnXkFtZTcwMTU0NTIzMw@@._V1_.jpg', 'genre': ['Action', 'Adventure', 'Sci-Fi'], 'contentRating': 'PG-13', 'actor': [{'@type': 'Person', 'url': '/name/nm0000375/', 'name': 'Robert Downey Jr.'}, {'@type': 'Person', 'url': '/name/nm0000569/', 'name': 'Gwyneth Paltrow'}, {'@type': 'Person', 'url': '/name/nm0005024/', 'name': 'Terrence Howard'}, {'@type': 'Person', 'url': '/name/nm0000313/', 'name': 'Jeff Bridges'}], 'director': {'@type': 'Person', 'url': '/name/nm0269463/', 'name': 'Jon Favreau'}, 'creator': [{'@type': 'Person', 'url': '/name/nm1318843/', 'name': 'Mark Fergus'}, {'@type': 'Person', 'url': '/name/nm1319757/', 'name': 'Hawk Ostby'}, {'@type': 'Person', 'url': '/name/nm1436466/', 'name': 'Art Marcum'}, {'@type': 'Person', 'url': '/name/nm0391344/', 'name': 'Matt Ho

In [5]:
movie_collection.count_documents({})

205951